In [8]:
# magic incantation to make the notebook wider
from IPython.core.display import HTML
HTML("<style>.container { width:90% !important; }</style>")

In [2]:
#magic incantation to make all text in LaTeX font:
from matplotlib import rc
# rc('font',**{'family':'serif','serif':['Computer Modern Roman']})
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('text', usetex=True)

In [3]:
import numpy as np
import pylab as pl
import os, sys
%matplotlib inline

import new_functions as fn
fn = reload(fn)

# XMAX = 200
# XMIN = 50
# YMAX = 200
# YMIN = 50

XMAX = 256
XMIN = 0
YMAX = 256
YMIN = 0

/Users/mfisherlevine/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [4]:
timepix_data_dir_ir = '/Users/mfisherlevine/Downloads/160802_beads1/'
# timepix_data_dir_ir = '/Users/mfisherlevine/Downloads/test/'


dirs = [timepix_data_dir_ir + _ for _ in os.listdir(timepix_data_dir_ir) if _.find('.DS')==-1]
tp_datafiles_ir = []
for this_dir in dirs:
    tp_datafiles_ir.extend([os.path.join(this_dir, _) for _ in os.listdir(this_dir) if _.find('.DS')==-1])
print 'Found %s datafiles in run %s'%(len(tp_datafiles_ir), timepix_data_dir_ir)


Found 100000 datafiles in run /Users/mfisherlevine/Downloads/160802_beads1/


In [ ]:
tp_data_ir = {}
n_files_loaded = 0
for i, filename in enumerate(tp_datafiles_ir):
    if i%1000==0: print 'Processed %s files'%i; sys.stdout.flush()
    n_codes = fn.Get_n_timecodesFromFile(filename)
    if not filename in tp_data_ir.keys():
        tp_data_ir[filename] = {}
        tp_data_ir[filename]['n_codes'] = n_codes
        tp_data_ir[filename]['filename'] = filename
    else:
        print 'Would have overwritten bunchID %s'%bunchID
    n_files_loaded += 1

print '*****\nLoaded %s files' %n_files_loaded



In [5]:
def SmartCentroid(filelist, use_CoM_as_centroid=True, inc_diagnoal_joins=False, bands=None, peak_range=20, DEBUG=0, skiplines=0, files_have_bunchIDs=False, n_tof_files=50, ToF_noise_threshold=0):
    '''Get smart x,y,t centroids from a fileset. A ToF spectrum is generated
    using a subset of the files (evenly sampled from the set)
    to set the band boundaries, though they can be manually specified as well.
    If use_CoM_as_centroid==False, the earliest pixel in the cluster is used,
    otherwise the centre of mass of the cluster is taken as the centroid.'''
    import scipy.ndimage as ndimage
    from scipy.ndimage.measurements import center_of_mass, maximum_position
    from scipy.signal import argrelmax, savgol_filter
    import sys
    import numpy as np
    
    ret = {} #return structure.
    
    if not bands:
        # build a set of files to use for making the ToF
        # ToF can chage during delay scans etc, so good to pull from across the range
        # files rather than just taking the first n files
        n_files = len(filelist)
        tof_fileset = []
        if n_tof_files >= n_files: #if there fewer files that the nominal, take them all
            tof_fileset = filelist
        else: 
            indices = [int(np.ceil(i * n_files / n_tof_files)) for i in xrange(n_tof_files)] #evenly sample the files
            if DEBUG >2:print 'Selecting file nums for ToF: %s'%indices
            for i in indices:
                tof_fileset.append(filelist[i])
            if DEBUG >1: print 'selected %s files for ToF'%len(tof_fileset);sys.stdout.flush()

        tof_imgs = []
        for filename in tof_fileset: # load images
            tof_imgs.append(fn.TimepixFileToImage(filename, skiplines=skiplines))
        maxval = int(np.max(tof_imgs)) # get ranges for histogram
        minval = int(np.min([np.min(_[_>0]) for _ in tof_imgs])) 

        ys = np.zeros(((maxval-minval)+1,), dtype=np.int64)
        xs = np.linspace(minval,maxval,(maxval-minval+1)) # make x points for ToF plot

#         for filename in tof_fileset: # build ToF histogram from ToF files
        for img in tof_imgs: # histogram each imgage
            ys += ndimage.histogram(img[img>0],minval,maxval,bins = (maxval-minval)+1) #much faster than pl.hist
        
        ys[ys<ToF_noise_threshold] = 0 #filter very small bins
        # redo the ranges after filtering out the small values:
        first_value = 0
        point = 0
        while ys[point]<=0:
            point += 1
            first_value += 1
        
        new_ys = savgol_filter(ys,5,3) # smooth the ToF spectrum
        max_indexes = argrelmax(new_ys, axis=0, order=peak_range) # find local maxima, range of 5 each side
        peaks = [xs[_] for _ in max_indexes[0]] # Get peak location from indices

#         print xs
#         return
        
        bands = [] # generate banks from peaks - need to take the midpoints though!
        bands.append((minval,(peaks[0]+peaks[1])//2)) #first point to midpoint of first peaks
        for i in xrange(1,len(peaks)-1):
            bands.append((bands[-1][1],(peaks[i]+peaks[i+1])//2)) # loop through
        bands.append((bands[-1][0],maxval))# add last midpoint to last value

        if DEBUG>0:
            f = pl.figure(figsize=[12,4]) # Make the figure an appropriate shape
            ax = pl.subplot(111)
            pl.plot(xs,ys,'b') #plot the original ToF
            pl.plot(xs,new_ys,'r')#plot the new, smoothed ToF
            pl.plot(peaks,[-10 for _ in max_indexes[0]],'bo') # plot peaks in blue
            pl.plot([_[0] for _ in bands],[10 for _ in bands],'ro') # plot left band boundaries in red
            pl.plot([_[1] for _ in bands],[10 for _ in bands],'ro') # plot right band boundaries in red too
#             ticks = [_[0] for _ in bands[1:]]
#             ax.set_xticks(ticks, minor=False)
#             ax.xaxis.grid(True, which='major')
            pl.xlim(minval,maxval) #set limits as python is weird sometimes

            # TODO: add a legend saying that colours are what
            if DEBUG >1:
                print 'Peaks at %s'%peaks
                print 'Bands set at %s'%bands;sys.stdout.flush()
            pl.show()
                  
            
    ################################################
    # We have our band, now let's find the clusters:
    if DEBUG >0: print 'Using %s band(s)...'%len(bands);sys.stdout.flush()

    if inc_diagnoal_joins:
        struct_el=[[1,1,1],[1,1,1],[1,1,1]] # for including diagonal connections as well
    else:
        struct_el=[[0,1,0],[1,1,1],[0,1,0]] # for vertical/horizontal connections only

    for filenum,filename in enumerate(filelist):
        if filenum%1000==0: print 'Centroided %s of %s files'%(filenum, len(filelist)); sys.stdout.flush()
        if files_have_bunchIDs: # if we're using bunchIDs then use these as keys in return dict
            fileID = fn.GetBunchIDFromFile(filename)
        else: #otherwise use the filenames
            fileID = filename
        ret[fileID]={'xs':[],'ys':[],'ts':[],'npixs':[]}

        img = fn.TimepixFileToImage(filename, skiplines=skiplines)

        segmentation, segments = ndimage.label(img, struct_el) # find clusters
        if DEBUG>2: print 'Found %s clusters without using band information'%segments;sys.stdout.flush()
        if DEBUG>3:
            print 'Segmented image:';sys.stdout.flush()
            f = pl.figure(figsize=[8,8]) 
            pl.imshow(segmentation)
            pl.show()

        seg_sum = 0
        for bandnum,(tmin, tmax) in enumerate(bands): # Process each band in turn
            band_img = img.copy() # make a copy
            band_img[band_img > tmax] = 0 # threshold the new image
            band_img[band_img <= tmin] = 0
            if DEBUG>2:
                print 'Band %s (%s - %s) image:'%(bandnum, tmin, tmax);sys.stdout.flush()
                f = pl.figure(figsize=[8,8]) 
                pl.imshow(band_img)
                pl.show()

            # find clusters
            segmentation, segments = ndimage.label(band_img, struct_el) 
            seg_sum += segments
            if DEBUG>3: print 'Found %s segs in band %s (%s - %s)'%(segments,i, tmin, tmax);sys.stdout.flush()
            
            # Get centroids from clusters using specified method:
            if use_CoM_as_centroid: # use center of mass weighting
                # TODO: check whether cluster 0 is included in CoMs
                # TODO: check order of these, i.e. if x,y,t will be coherent
                # NB must use segmentation image for CoM finding to avoid using timecodes as weigths (especially without first inverting!)
                CoMs = center_of_mass(segmentation, segmentation, [_ for _ in xrange(1,segments+1)])
                for clust_num in xrange(1,segments): # cluster 0 = background so skip it
                    clust_pix_index = np.where(segmentation==clust_num) # find pixels associated with cluster
                    ret[fileID]['ts'].append(np.max(img[clust_pix_index]))
                    ret[fileID]['xs'].append(CoMs[clust_num][0])
                    ret[fileID]['ys'].append(CoMs[clust_num][1])
                    ret[fileID]['npixs'].append(len(clust_pix_index[0]))

            else: # Take the earliest timecode in the cluster as the centroid
                CoMs = maximum_position(band_img, segmentation, [_ for _ in xrange(1,segments+1)])
                print 'WARNING!! Unsupported/unimplemented - what happens if multiple pixels have same timecode?!'
                for com in CoMs: # Stepping through together, so def. coherent
                    ret[fileID]['ts'].append(img[com])
                    ret[fileID]['xs'].append(com[0])
                    ret[fileID]['ys'].append(com[1])
                    ret[fileID]['npixs'].append('????')

            if DEBUG>2: print 'CoMs for band %s: %s'(bandnum, CoMs);sys.stdout.flush()
        #     index = (np.asarray([_[0] for _ in CoMs]),np.asarray([[_[1] for _ in CoMs]]))
        #     codes = img[CoMs]

        if DEBUG>0: print 'Found %s clusters when using bands'%seg_sum;sys.stdout.flush()

    print 'Finished centroiding %s files'%len(filelist)
    ########### Finished!
    if DEBUG>0:
        ts = []
        for key in ret.keys():
            ts.extend(ret[key]['ts'])
        print 'Centroided ToF Spectrum'
        f = pl.figure(figsize=[12,4]) #Make the figure an appropriate shape
        a,b,c = pl.hist(ts, max(ts)-min(ts)+1, histtype = 'step')
        pl.xlim(minval,maxval) #set limits as python is weird sometimes

    return ret

fn = reload(fn)
ret_dict = SmartCentroid(tp_datafiles_ir, DEBUG=0, n_tof_files=50, skiplines=1, files_have_bunchIDs=False, use_CoM_as_centroid=True, bands=[(0,11810)])
# SmartCentroid(tp_datafiles[1:509],DEBUG=1, skiplines=1, files_have_bunchIDs=True)

Centroided 0 of 100000 files


/Users/mfisherlevine/anaconda/lib/python2.7/site-packages/numpy/lib/npyio.py:893: UserWarning: loadtxt: Empty input file: "/Users/mfisherlevine/Downloads/160802_beads1/00/beads1_2016_08_02_09_31_08_199.txt"
  warnings.warn('loadtxt: Empty input file: "%s"' % fname)
/Users/mfisherlevine/anaconda/lib/python2.7/site-packages/numpy/lib/npyio.py:893: UserWarning: loadtxt: Empty input file: "/Users/mfisherlevine/Downloads/160802_beads1/00/beads1_2016_08_02_09_31_22_899.txt"
  warnings.warn('loadtxt: Empty input file: "%s"' % fname)
/Users/mfisherlevine/anaconda/lib/python2.7/site-packages/numpy/lib/npyio.py:893: UserWarning: loadtxt: Empty input file: "/Users/mfisherlevine/Downloads/160802_beads1/00/beads1_2016_08_02_09_31_34_913.txt"
  warnings.warn('loadtxt: Empty input file: "%s"' % fname)


Centroided 1000 of 100000 files


/Users/mfisherlevine/anaconda/lib/python2.7/site-packages/numpy/lib/npyio.py:893: UserWarning: loadtxt: Empty input file: "/Users/mfisherlevine/Downloads/160802_beads1/00/beads1_2016_08_02_09_31_55_557.txt"
  warnings.warn('loadtxt: Empty input file: "%s"' % fname)
/Users/mfisherlevine/anaconda/lib/python2.7/site-packages/numpy/lib/npyio.py:893: UserWarning: loadtxt: Empty input file: "/Users/mfisherlevine/Downloads/160802_beads1/00/beads1_2016_08_02_09_32_17_481.txt"
  warnings.warn('loadtxt: Empty input file: "%s"' % fname)
/Users/mfisherlevine/anaconda/lib/python2.7/site-packages/numpy/lib/npyio.py:893: UserWarning: loadtxt: Empty input file: "/Users/mfisherlevine/Downloads/160802_beads1/00/beads1_2016_08_02_09_32_43_247.txt"
  warnings.warn('loadtxt: Empty input file: "%s"' % fname)
/Users/mfisherlevine/anaconda/lib/python2.7/site-packages/numpy/lib/npyio.py:893: UserWarning: loadtxt: Empty input file: "/Users/mfisherlevine/Downloads/160802_beads1/00/beads1_2016_08_02_09_32_49_645.

Centroided 2000 of 100000 files


...

Centroided 98000 of 100000 files


/Users/mfisherlevine/anaconda/lib/python2.7/site-packages/numpy/lib/npyio.py:893: UserWarning: loadtxt: Empty input file: "/Users/mfisherlevine/Downloads/160802_beads1/02/beads1_2016_08_02_12_48_20_233.txt"
  warnings.warn('loadtxt: Empty input file: "%s"' % fname)
/Users/mfisherlevine/anaconda/lib/python2.7/site-packages/numpy/lib/npyio.py:893: UserWarning: loadtxt: Empty input file: "/Users/mfisherlevine/Downloads/160802_beads1/02/beads1_2016_08_02_12_49_11_524.txt"
  warnings.warn('loadtxt: Empty input file: "%s"' % fname)
/Users/mfisherlevine/anaconda/lib/python2.7/site-packages/numpy/lib/npyio.py:893: UserWarning: loadtxt: Empty input file: "/Users/mfisherlevine/Downloads/160802_beads1/02/beads1_2016_08_02_12_49_16_928.txt"
  warnings.warn('loadtxt: Empty input file: "%s"' % fname)
/Users/mfisherlevine/anaconda/lib/python2.7/site-packages/numpy/lib/npyio.py:893: UserWarning: loadtxt: Empty input file: "/Users/mfisherlevine/Downloads/160802_beads1/02/beads1_2016_08_02_12_49_37_924.

Centroided 99000 of 100000 files


/Users/mfisherlevine/anaconda/lib/python2.7/site-packages/numpy/lib/npyio.py:893: UserWarning: loadtxt: Empty input file: "/Users/mfisherlevine/Downloads/160802_beads1/02/beads1_2016_08_02_12_50_55_707.txt"
  warnings.warn('loadtxt: Empty input file: "%s"' % fname)
/Users/mfisherlevine/anaconda/lib/python2.7/site-packages/numpy/lib/npyio.py:893: UserWarning: loadtxt: Empty input file: "/Users/mfisherlevine/Downloads/160802_beads1/02/beads1_2016_08_02_12_51_02_910.txt"
  warnings.warn('loadtxt: Empty input file: "%s"' % fname)
/Users/mfisherlevine/anaconda/lib/python2.7/site-packages/numpy/lib/npyio.py:893: UserWarning: loadtxt: Empty input file: "/Users/mfisherlevine/Downloads/160802_beads1/02/beads1_2016_08_02_12_51_13_607.txt"
  warnings.warn('loadtxt: Empty input file: "%s"' % fname)
/Users/mfisherlevine/anaconda/lib/python2.7/site-packages/numpy/lib/npyio.py:893: UserWarning: loadtxt: Empty input file: "/Users/mfisherlevine/Downloads/160802_beads1/02/beads1_2016_08_02_12_51_25_738.

Finished centroiding 100000 files


In [ ]:
fn = reload(fn)

# fn.ShowClusteredImage('/Users/mfisherlevine/Downloads/160721_voltages1/160721_v9/160721_v9_0250.txt',257)
fn.ShowClusteredImage(tp_datafiles_ir[4],257,skiplines=1)

In [7]:
all_ts = []
output_list = []
output_list_mult = []

# imshow_size = 10
multiplier = 100
width = 2.0
img = np.zeros((256,256), dtype=np.float64)
for filenum, filename in enumerate(ret_dict.keys()):
    if filenum%1000==0:
        WriteOutput('/Users/mfisherlevine/Downloads/test/160802_beads1.txt',output_list)
        WriteOutput('/Users/mfisherlevine/Downloads/test/160802_beads1_mult.txt',output_list_mult)

        print 'Gaussianised %s files'%filenum; sys.stdout.flush()
        output_list = []
        output_list_mult = []

    for x, y, t, npix in zip(ret_dict[filename]['xs'],ret_dict[filename]['ys'],ret_dict[filename]['ts'],ret_dict[filename]['npixs']):
        if npix<4: continue
#         all_ts.append(11810-t)
#         print x,y,t
        img = fn.makeGaussian(256,multiplier,width,[x,y])
        indices = np.where(img>=1)
#         print 'npix above 1: %s'%len(indices[0])
#         print np.sum(img)
        for xval, yval in zip(indices[0],indices[1]):
            mult = int(np.round(img[xval,yval]))
            _ = '%s\t%s\t%s\t%s\n'%(xval,yval,int(11810-t),mult)
            output_list_mult.append(_)
            _ = '%s\t%s\t%s\n'%(xval,yval,int(11810-t))
            for i in xrange(mult):
                output_list.append(_)
#         pl.imshow(img[y-imshow_size:y+imshow_size,x-imshow_size:x+imshow_size], interpolation='nearest')
#         break
#     break

WriteOutput('/Users/mfisherlevine/Downloads/test/160802_beads1.txt',output_list)
WriteOutput('/Users/mfisherlevine/Downloads/test/160802_beads1_mult.txt',output_list_mult)

Gaussianised 0 files
Gaussianised 1000 files
Gaussianised 2000 files
Gaussianised 3000 files
Gaussianised 4000 files
Gaussianised 5000 files
Gaussianised 6000 files
Gaussianised 7000 files
Gaussianised 8000 files
Gaussianised 9000 files
Gaussianised 10000 files
Gaussianised 11000 files
Gaussianised 12000 files
Gaussianised 13000 files
Gaussianised 14000 files
Gaussianised 15000 files
Gaussianised 16000 files
Gaussianised 17000 files
Gaussianised 18000 files
Gaussianised 19000 files
Gaussianised 20000 files
Gaussianised 21000 files
Gaussianised 22000 files
Gaussianised 23000 files
Gaussianised 24000 files
Gaussianised 25000 files
Gaussianised 26000 files
Gaussianised 27000 files
Gaussianised 28000 files
Gaussianised 29000 files
Gaussianised 30000 files
Gaussianised 31000 files
Gaussianised 32000 files
Gaussianised 33000 files
Gaussianised 34000 files
Gaussianised 35000 files
Gaussianised 36000 files
Gaussianised 37000 files
Gaussianised 38000 files
Gaussianised 39000 files
Gaussianised 

In [ ]:
# output_list = ['a','b']

WriteOutput('/Users/mfisherlevine/Downloads/test/output.txt',output_list)

In [ ]:
print len(output_list)

In [6]:
def WriteOutput(filename, outlist):
    outfile = open(filename, mode='a')
    for line in outlist:
        outfile.write(line)
    outfile.close()

In [ ]:
# range = [0,11810]
range = [1800,1900]

############
# for the raw spectrum:

# y_rescale = 1.1 # big peak is too big, so clip y-axis by rescaling to this fraction of max
# bins = min(int((max(raw_timecodes_ir)-min(raw_timecodes_ir))),(range[1]-range[0]))
# # print bins
# fig = pl.figure(figsize=[16,4])
# ax = pl.subplot(111)

# # as a hist
# print 'Raw timecodes a histogram:'
# ys_ir, binEdges_ir, dummy1 = pl.hist(raw_timecodes_ir, bins=bins, range=range, histtype = 'step')
# # bincenters = 0.5*(binEdges[1:]+binEdges[:-1])
# lims = pl.ylim()
# pl.ylim([lims[0], y_rescale*max(ys_ir)])
# # ax.set_xticks(ticks, minor=False)
# # turn_raw, stat_raw, troughs_raw, peaks_raw = fn.GetTurningPoints(ys, bincenters, noise=20)
# # ax.set_xticks(troughs_raw, minor=False)
# ax.xaxis.grid(True, which='major')
# pl.xlim(range[0],range[1])
# pl.show()


############
# for the centroided spectrum:

y_rescale = 1.1 # big peak is too big, so clip y-axis by rescaling to this fraction of max
bins = min(int((max(all_ts)-min(all_ts)+1)),(range[1]-range[0]+1))

fig = pl.figure(figsize=[16,4])
ax = pl.subplot(111)

# as a hist
print 'As a histogram:'
ys_ir, binEdges_ir, dummy1 = pl.hist(all_ts, bins=bins, range=range, histtype = 'step')

# # bincenters = 0.5*(binEdges[1:]+binEdges[:-1])
# # turn_cent, stat_cent, troughs_cent, peaks_cent = fn.GetTurningPoints(ys, bincenters, noise=1)
# lims = pl.ylim()
# pl.xlim(range[0],range[1])
# pl.ylim([lims[0], y_rescale*max(ys_ir)])
# # ax.set_xticks(troughs_cent, minor=False)
# ax.xaxis.grid(True, which='major')
# pl.show()

# # as a hist
# fig = pl.figure(figsize=[16,4])
# ax = pl.subplot(111)
# print 'With 4 pixel threshold:'
# ys_ir, binEdges_ir, dummy1 = pl.hist(all_ts_4pix_ir, bins=bins, range=range, histtype = 'step')

# # bincenters = 0.5*(binEdges[1:]+binEdges[:-1])
# # turn_cent, stat_cent, troughs_cent, peaks_cent = fn.GetTurningPoints(ys, bincenters, noise=0)
# lims = pl.ylim()
# pl.xlim(range[0],range[1])
# pl.ylim([lims[0], y_rescale*max(ys_ir)])
# # ax.set_xticks(troughs_cent, minor=False)
# ax.xaxis.grid(True, which='major')
# pl.show()


In [ ]:
# Load the raw timecodes, converting the same way as the others
fn = reload(fn)

raw_timecodes_ir = []
for i, filename in enumerate(tp_datafiles_ir):
    if i%1000==0: print 'Loaded %s files'%i; sys.stdout.flush()
#     raw_timecodes_ir.extend(((11810-_)-TZERO) for _ in fn.GetTimecodes_SingleFile(filename, skiplines=0))
    raw_timecodes_ir.extend(((11810-_)-TZERO) for _ in fn.GetTimecodes_SingleFile(filename, skiplines=1,winow_xmax=XMAX,winow_xmin=XMIN,winow_ymax=YMAX,winow_ymin=YMIN))
print 'IR: Loaded %s raw timecodes'%len(raw_timecodes_ir)


In [ ]:
# range = [900,1200]
range = [1800,1900]

############
# for the raw spectrum:

y_rescale = 1.1 # big peak is too big, so clip y-axis by rescaling to this fraction of max
bins = min(int((max(raw_timecodes_ir)-min(raw_timecodes_ir))),(range[1]-range[0]))
# print bins
fig = pl.figure(figsize=[16,4])
ax = pl.subplot(111)

# as a hist
print 'Raw timecodes a histogram:'
ys_ir, binEdges_ir, dummy1 = pl.hist(raw_timecodes_ir, bins=bins, range=range, histtype = 'step')
# bincenters = 0.5*(binEdges[1:]+binEdges[:-1])
lims = pl.ylim()
pl.ylim([lims[0], y_rescale*max(ys_ir)])
# ax.set_xticks(ticks, minor=False)
# turn_raw, stat_raw, troughs_raw, peaks_raw = fn.GetTurningPoints(ys, bincenters, noise=20)
# ax.set_xticks(troughs_raw, minor=False)
ax.xaxis.grid(True, which='major')
pl.xlim(range[0],range[1])
pl.show()


############
# for the centroided spectrum:

y_rescale = 1.1 # big peak is too big, so clip y-axis by rescaling to this fraction of max
# bins = min(int((max(all_ts)-min(all_ts)+1)),(range[1]-range[0]+1))

fig = pl.figure(figsize=[16,4])
ax = pl.subplot(111)

# as a hist
print 'As a histogram:'
ys_ir, binEdges_ir, dummy1 = pl.hist(all_ts, bins=bins, range=range, histtype = 'step')

# bincenters = 0.5*(binEdges[1:]+binEdges[:-1])
# turn_cent, stat_cent, troughs_cent, peaks_cent = fn.GetTurningPoints(ys, bincenters, noise=1)
lims = pl.ylim()
pl.xlim(range[0],range[1])
pl.ylim([lims[0], y_rescale*max(ys_ir)])
# ax.set_xticks(troughs_cent, minor=False)
ax.xaxis.grid(True, which='major')
pl.show()

# as a hist
fig = pl.figure(figsize=[16,4])
ax = pl.subplot(111)
print 'With 4 pixel threshold:'
ys_ir, binEdges_ir, dummy1 = pl.hist(all_ts_4pix_ir, bins=bins, range=range, histtype = 'step')

# bincenters = 0.5*(binEdges[1:]+binEdges[:-1])
# turn_cent, stat_cent, troughs_cent, peaks_cent = fn.GetTurningPoints(ys, bincenters, noise=0)
lims = pl.ylim()
pl.xlim(range[0],range[1])
pl.ylim([lims[0], y_rescale*max(ys_ir)])
# ax.set_xticks(troughs_cent, minor=False)
ax.xaxis.grid(True, which='major')
pl.show()


In [ ]:
for t, y in zip(binEdges_ir,ys_ir):
    print t, y

In [ ]:
bands = []
# # for i in xrange(len(troughs_cent)-1):
# #     bands.append((troughs_cent[i],troughs_cent[i+1]))
# # print bands
# bands.append([0,250])

# bands.append([900,910])
# bands.append([910,920])
# bands.append([920,930])
# bands.append([930,940])
# bands.append([940,950])
# bands.append([950,1000])
# bands.append([1000,1200])

bands.append([1800,1820])
bands.append([1820,1860])


In [ ]:
#probably don't need to rerun this, and it takes ages, so don't do it by accident:
if True: 
    import time
    now = time.time()
    n_bunches = 999999

    images_ir = [np.zeros((256,256), dtype = np.float64) for _ in bands]
    for i, t_range in enumerate(bands):
        print 'Processing band %s of %s'%(i+1, len(bands))
        for bunchID in sorted(tp_data_ir.keys())[:min(n_bunches,tp_data_ir.keys())]:
            if 'xs' not in tp_data_ir[bunchID]: continue
            for x,y,t,npix in zip(tp_data_ir[bunchID]['xs'],
                                  tp_data_ir[bunchID]['ys'],
                                  tp_data_ir[bunchID]['ts'],
                                  tp_data_ir[bunchID]['npixs']):
                if t >= t_range[0] and t<t_range[1]:
                    images_ir[i] += fn.makeGaussian(256,1,1,[x,y])
#                     images_ir[i] += fn.makeGaussian(256,1,(npix**.5)/1.5,[x,y])

    print 'Took %.1f secs'%(time.time()-now)


In [ ]:
#### pickle the bands and their images
pickle_filename = '/Users/mfisherlevine/Desktop/desy/pickles/run'+run_id+'_VMIs.pickle'

if False: #C areful, this will overwrite the good pickles if you accidentally run it!
    pickle_file = open(pickle_filename, 'wb')
    pickle.dump([bands, images], pickle_file)
    pickle_file.close()


#### Load the bands and their images from the pickle:
if True:
    print 'Unpickling...'; sys.stdout.flush()
    pickle_file = open(pickle_filename, 'rb')
    [bands, images] = pickle.load(pickle_file)
    pickle_file.close()
    print 'Loaded bands and images'

In [ ]:
fn = reload(fn)
cmap = 'gray' ### beware using the de facto standard 'jet',

### but it's also replotted below for people who can't live without it
for i, image in enumerate(images_ir):
    title = 'IR image for times %s to %s ns(band #%s)'%(bands[i][0],bands[i][1], i)
    fn.DisplayImage(image, cmap=cmap, title=title,vmax='auto',savefig='/Users/mfisherlevine/Desktop/new_USAF_IR_'+str(i)+'.pdf')
    pl.show()

    

In [ ]:
fn = reload(fn)
cmap = 'jet'
for i, image in enumerate(images_ir):
    title = 'VMI for times %s to %s (band %s)'%(bands[i][0],bands[i][1], i)
    fn.DisplayImage(image, cmap=cmap, title=title,vmax='auto')
    pl.show()